In [6]:
from pymongo import MongoClient
from datetime import datetime

# Conexão com o MongoDB

In [7]:
MONGO_URI = "mongodb://0.0.0.0:27017"
client = MongoClient(MONGO_URI)

# Seleção do banco de dados e coleção

In [8]:
db = client["workshop_db"]
collection = db["workshop_collection"]

# Inserindo dados

In [19]:
documents = [
        {
            "name": "Alice",
            "age": 25,
            "city": "São Paulo",
            "hobbies": ["reading", "traveling"],
            "purchases": [
                {"item": "book", "price": 30, "date": datetime(2024, 12, 10)},
                {"item": "laptop", "price": 5000, "date": datetime(2024, 11, 5)},
            ],
        },
        {
            "name": "Bob",
            "age": 30,
            "city": "Rio de Janeiro",
            "hobbies": ["sports", "music"],
            "purchases": [
                {"item": "guitar", "price": 1500, "date": datetime(2024, 10, 15)},
                {"item": "headphones", "price": 200, "date": datetime(2024, 12, 1)},
            ],
        },
        {
            "name": "Charlie",
            "age": 35,
            "city": "Belo Horizonte",
            "hobbies": ["photography", "traveling"],
            "purchases": [
                {"item": "camera", "price": 3500, "date": datetime(2024, 8, 20)},
            ],
        },
        {
            "name": "Diana",
            "age": 28,
            "city": "Curitiba",
            "hobbies": ["cooking", "dancing"],
            "purchases": [
                {"item": "mixer", "price": 300, "date": datetime(2024, 9, 10)},
                {"item": "shoes", "price": 150, "date": datetime(2024, 10, 1)},
            ],
        },
    ]
result = collection.insert_many(documents)

In [20]:
print(f"Documentos inseridos: {result.inserted_ids}")

Documentos inseridos: [ObjectId('6761d5ddb8ade5c5e16959d8'), ObjectId('6761d5ddb8ade5c5e16959d9'), ObjectId('6761d5ddb8ade5c5e16959da'), ObjectId('6761d5ddb8ade5c5e16959db')]


# Consulta campos aninhados e arrays.

In [21]:
print("\nUsuários com hobbies que incluem 'traveling':")
for doc in collection.find({"hobbies": "traveling"}):
    print(doc)


Usuários com hobbies que incluem 'traveling':
{'_id': ObjectId('6761d5c1b8ade5c5e16959d4'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6761d5c1b8ade5c5e16959d6'), 'name': 'Charlie', 'age': 35, 'city': 'Belo Horizonte', 'hobbies': ['photography', 'traveling'], 'purchases': [{'item': 'camera', 'price': 3500, 'date': datetime.datetime(2024, 8, 20, 0, 0)}]}
{'_id': ObjectId('6761d5ddb8ade5c5e16959d8'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6761d5ddb8ade5c5e16959da'), 'name': 'Charlie', 'age': 35, 'city': 'Belo Horizonte', 

# Exemplo avançado de agregação.

In [22]:
pipeline = [
    {"$unwind": "$purchases"},  # Expande o array de compras
    {"$group": {
        "_id": "$name",
        "total_spent": {"$sum": "$purchases.price"},
        "average_spent": {"$avg": "$purchases.price"},
        "max_spent": {"$max": "$purchases.price"},
    }},
    {"$sort": {"total_spent": -1}},  # Ordena pelo total gasto (decrescente)
]

print("\nAgregação avançada (gastos totais por usuário):")
for result in collection.aggregate(pipeline):
    print(result)


Agregação avançada (gastos totais por usuário):
{'_id': 'Alice', 'total_spent': 10060, 'average_spent': 2515.0, 'max_spent': 5000}
{'_id': 'Bob', 'total_spent': 8500, 'average_spent': 850.0, 'max_spent': 1500}
{'_id': 'Charlie', 'total_spent': 7000, 'average_spent': 3500.0, 'max_spent': 3500}
{'_id': 'Diana', 'total_spent': 2250, 'average_spent': 225.0, 'max_spent': 300}


# Consultas específicas com arrays.

In [23]:
print("\nUsuários com mais de uma compra:")
for doc in collection.find({"$expr": {"$gte": [{"$size": "$purchases"}, 2]}}):
    print(doc)

print("\nUsuários que compraram algo em Outubro de 2024:")
for doc in collection.find({"purchases.date": {
    "$gte": datetime(2024, 10, 1),
    "$lt": datetime(2024, 11, 1),
}}):
    print(doc)


Usuários com mais de uma compra:
{'_id': ObjectId('6761d444bdca8c00e24a1315'), 'name': 'Bob', 'age': 30, 'city': 'Rio de Janeiro', 'hobbies': ['sports', 'music'], 'purchases': [{'item': 'guitar', 'price': 1500, 'date': datetime.datetime(2024, 10, 15, 0, 0)}, {'item': 'headphones', 'price': 200, 'date': datetime.datetime(2024, 12, 1, 0, 0)}]}
{'_id': ObjectId('6761d444bdca8c00e24a1317'), 'name': 'Diana', 'age': 28, 'city': 'Curitiba', 'hobbies': ['cooking', 'dancing'], 'purchases': [{'item': 'mixer', 'price': 300, 'date': datetime.datetime(2024, 9, 10, 0, 0)}, {'item': 'shoes', 'price': 150, 'date': datetime.datetime(2024, 10, 1, 0, 0)}]}
{'_id': ObjectId('6761d519b8ade5c5e16959cd'), 'name': 'Bob', 'age': 30, 'city': 'Rio de Janeiro', 'hobbies': ['sports', 'music'], 'purchases': [{'item': 'guitar', 'price': 1500, 'date': datetime.datetime(2024, 10, 15, 0, 0)}, {'item': 'headphones', 'price': 200, 'date': datetime.datetime(2024, 12, 1, 0, 0)}]}
{'_id': ObjectId('6761d519b8ade5c5e16959cf

# Atualizações em campos específicos e arrays.

In [24]:
result = collection.update_one(
        {"name": "Alice"},
        {"$push": {"purchases": {"item": "phone", "price": 2000, "date": datetime(2024, 12, 15)}}}
    )
print(f"\nDocumentos atualizados: {result.modified_count}")


Documentos atualizados: 1


# Exclusão baseada em critérios complexos.

In [25]:
result = collection.delete_many({"purchases.price": {"$gt": 3000}})
print(f"\nDocumentos excluídos: {result.deleted_count}")


Documentos excluídos: 4
